In [ ]:
!pip install muda
!pip install jams
!sudo apt-get install sox

In [ ]:
# Edit file in /usr/local/lib/python3.11/dist-packages/jams/schema.py  to use np.float_ = np.float64
import os
import librosa
import jams
import muda

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
input_root = "/content/drive/MyDrive/clipped_data"
output_root = "/content/drive/MyDrive/augmented_data/drc"

# Initialize the DRC augmenter once (outside the loop)
drc = muda.deformers.DynamicRangeCompression(preset="music standard")

for subdir, _, files in os.walk(input_root):
    for file in files:
        if file.endswith('.wav'):
            input_path = os.path.join(subdir, file)
            rel_path = os.path.relpath(subdir, input_root)
            output_dir = os.path.join(output_root, rel_path)
            os.makedirs(output_dir, exist_ok=True)
            output_path = os.path.join(output_dir, file)

            # === Load audio ===
            y, sr = librosa.load(input_path, sr=None)

            # Create an empty JAMS object (needed by muda)
            jam = jams.JAMS()
            jam.file_metadata.duration = librosa.get_duration(y=y, sr=sr)
            jam = muda.jam_pack(jam, _audio=dict(y=y, sr=sr))

            # === Apply DRC ===
            for _, jam_out in enumerate(drc.transform(jam)):
                muda.save(output_path, output_path.replace(".wav", ".jams"), jam_out)

In [ ]:
# will need to edit more library functions to accomodate updated librosa

input_root = "/content/drive/MyDrive/clipped_data"
output_root = "/content/drive/MyDrive/augmented_data/bg_noise"
background_noise = "quiet_clapping.wav"

# Initialize the background noise augmenter once (outside the loop)
bg_noise = muda.deformers.BackgroundNoise(n_samples=1, files=background_noise, weight_min=0.1, weight_max=0.5)

for subdir, _, files in os.walk(input_root):
    for file in files:
        if file.endswith('.wav'):
            input_path = os.path.join(subdir, file)
            rel_path = os.path.relpath(subdir, input_root)
            output_dir = os.path.join(output_root, rel_path)
            os.makedirs(output_dir, exist_ok=True)
            output_path = os.path.join(output_dir, file)

            # === Load audio ===
            y, sr = librosa.load(input_path, sr=None)

            # Create an empty JAMS object (needed by muda)
            jam = jams.JAMS()
            jam.file_metadata.duration = librosa.get_duration(y=y, sr=sr)
            jam = muda.jam_pack(jam, _audio=dict(y=y, sr=sr))

            # === Apply Background Noise ===
            for _, jam_out in enumerate(bg_noise.transform(jam)):
                muda.save(output_path, output_path.replace(".wav", ".jams"), jam_out)